<h1>Foundations Of Robotics Final Project</h1>

In [ ]:
#comment this line below if packages are already installed
!pip install ipykernel ipython mujoco matplotlib pillow scipy

# import statements
import mujoco
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import Image as IPImage, display
import tempfile
import xml.etree.ElementTree as ET
import scipy

print("MuJoCo version:", mujoco.__version__)

<h1>Setup and Helper Functions (Do Not Modify)</h1>

In [ ]:
# Load model
model_path = os.path.join("mujoco_menagerie", "universal_robots_ur10e", "scene.xml")
model = mujoco.MjModel.from_xml_path(model_path)
data = mujoco.MjData(model)

print(f"Model loaded successfully!")
print(f"Number of joints (nq): {model.nq}")
print(f"Number of actuators: {model.nu}")
print(f"Timestep: {model.opt.timestep} seconds")

# Reset simulation
mujoco.mj_resetData(model, data)

# Camera settings
camera = mujoco.MjvCamera()
camera.distance = 3.5
camera.azimuth = 100
camera.elevation = -30
camera.lookat[:] = [0.0, 0.0, 0.5]

# Simulation parameters
duration = 5.0  # seconds
fps = 15

# Animation parameters (for velocity control)
animation_duration = 5.0
animation_fps = 15
arrow_scale = 1.0

# PD Controller Class
class PDController:
    def __init__(self, kp, kd, num_joints=6):
        self.kp = np.full(num_joints, kp) if np.isscalar(kp) else np.array(kp)
        self.kd = np.full(num_joints, kd) if np.isscalar(kd) else np.array(kd)
        self.target_angles = np.zeros(num_joints)
        
    def set_target(self, joint_idx, angle):
        self.target_angles[joint_idx] = angle
        
    def set_all_targets(self, angles):
        self.target_angles = np.array(angles)
        
    def compute_control(self, current_pos, current_vel):
        error = self.target_angles - current_pos
        error_derivative = -current_vel
        torque = self.kp * error + self.kd * error_derivative
        return torque

# Velocity PD Controller
class VelocityController:
    def __init__(self, num_joints=6):
        self.kd = np.array([800.0, 800.0, 800.0, 200.0, 200.0, 200.0])
        self.kp = np.array([500.0, 500.0, 500.0, 200.0, 200.0, 200.0])
        self.target_velocities = np.zeros(num_joints)
        self.initial_positions = np.zeros(num_joints)
        
    def set_target_velocities(self, velocities):
        self.target_velocities = np.array(velocities)
    
    def set_initial_positions(self, positions):
        self.initial_positions = np.array(positions)
    
    def compute_control(self, model, data):
        current_pos = data.qpos[:6]
        current_vel = data.qvel[:6]
        all_zero = np.allclose(self.target_velocities, 0.0, atol=1e-6)
        
        if all_zero:
            pos_error = self.initial_positions - current_pos
            vel_error = -current_vel
            data_copy = mujoco.MjData(model)
            data_copy.qpos[:] = data.qpos[:]
            data_copy.qvel[:] = 0.0
            data_copy.qacc[:] = 0.0
            mujoco.mj_inverse(model, data_copy)
            passive_forces = data_copy.qfrc_inverse[:6].copy()
            control_torque = passive_forces + self.kp * pos_error + self.kd * vel_error
        else:
            vel_error = self.target_velocities - current_vel
            data_copy = mujoco.MjData(model)
            data_copy.qpos[:] = data.qpos[:]
            data_copy.qvel[:] = data.qvel[:]
            data_copy.qacc[:] = 0
            mujoco.mj_inverse(model, data_copy)
            passive_forces = data_copy.qfrc_inverse[:6].copy()
            control_torque = passive_forces + self.kd * vel_error
        
        return control_torque

# Initialize PD controller
kp = 150.0
kd = 20.0
controller = PDController(kp, kd, num_joints=6)

print(f"\nPD Controller initialized (kp={kp}, kd={kd})")
print(f"Duration: {duration}s, FPS: {fps}")

# Helper Functions
def create_model_with_ball(original_xml_path, ball_position=[0.5, 0.5, 0.5], ball_radius=0.05):
    """Create a modified MuJoCo model with a red ball at specified xyz coordinates."""
    tree = ET.parse(original_xml_path)
    root = tree.getroot()
    
    worldbody = root.find('worldbody')
    if worldbody is None:
        worldbody = ET.SubElement(root, 'worldbody')
    
    ball_body = ET.SubElement(worldbody, 'body', {
        'name': 'red_ball',
        'pos': f"{ball_position[0]} {ball_position[1]} {ball_position[2]}"
    })
    
    ET.SubElement(ball_body, 'geom', {
        'name': 'ball_geom',
        'type': 'sphere',
        'size': str(ball_radius),
        'rgba': '1 0 0 1',
        'contype': '0',
        'conaffinity': '0'
    })
    
    original_dir = os.path.dirname(original_xml_path)
    temp_path = os.path.join(original_dir, 'scene_with_ball.xml')
    tree.write(temp_path, encoding='unicode')
    model = mujoco.MjModel.from_xml_path(temp_path)
    os.unlink(temp_path)
    
    return model

def create_model_with_velocity_arrow(original_xml_path, ee_pos, velocity_guess, arrow_scale=1.0):
    """Create a modified MuJoCo model with a cyan arrow showing the guessed velocity direction."""
    tree = ET.parse(original_xml_path)
    root = tree.getroot()
    
    worldbody = root.find('worldbody')
    if worldbody is None:
        worldbody = ET.SubElement(root, 'worldbody')
    
    vel_magnitude = np.linalg.norm(velocity_guess)
    if vel_magnitude > 1e-6:
        arrow_direction = np.array(velocity_guess) / vel_magnitude
        arrow_length = vel_magnitude * arrow_scale
        arrow_end = ee_pos + arrow_direction * arrow_length
        
        arrow_body = ET.SubElement(worldbody, 'body', {'name': 'velocity_arrow', 'pos': '0 0 0'})
        
        shaft_length = arrow_length * 0.8
        shaft_end = ee_pos + arrow_direction * shaft_length
        fromto_shaft = f"{ee_pos[0]} {ee_pos[1]} {ee_pos[2]} {shaft_end[0]} {shaft_end[1]} {shaft_end[2]}"
        
        ET.SubElement(arrow_body, 'geom', {
            'name': 'arrow_shaft',
            'type': 'capsule',
            'fromto': fromto_shaft,
            'size': '0.02',
            'rgba': '0 1 1 0.8',
            'contype': '0',
            'conaffinity': '0'
        })
        
        fromto_head = f"{shaft_end[0]} {shaft_end[1]} {shaft_end[2]} {arrow_end[0]} {arrow_end[1]} {arrow_end[2]}"
        
        ET.SubElement(arrow_body, 'geom', {
            'name': 'arrow_head',
            'type': 'capsule',
            'fromto': fromto_head,
            'size': '0.04',
            'rgba': '0 1 1 0.8',
            'contype': '0',
            'conaffinity': '0'
        })
    
    original_dir = os.path.dirname(original_xml_path)
    temp_path = os.path.join(original_dir, 'scene_with_arrow.xml')
    tree.write(temp_path, encoding='unicode')
    model = mujoco.MjModel.from_xml_path(temp_path)
    os.unlink(temp_path)
    
    return model

def run_position_control_simulation(model, controller, duration_sec, fps_val, camera_obj):
    """Run a position control simulation and return frames."""
    data = mujoco.MjData(model)
    mujoco.mj_resetData(model, data)
    renderer = mujoco.Renderer(model, height=480, width=640)
    
    timestep = model.opt.timestep
    frame_skip = int(1 / (timestep * fps_val))
    total_steps = int(duration_sec / timestep)
    
    frames = []
    for step in range(total_steps):
        current_pos = data.qpos[:6]
        current_vel = data.qvel[:6]
        control_torques = controller.compute_control(current_pos, current_vel)
        data.ctrl[:] = control_torques
        mujoco.mj_step(model, data)
        
        if step % frame_skip == 0:
            renderer.update_scene(data, camera=camera_obj)
            frames.append(Image.fromarray(renderer.render().copy()))
    
    renderer.close()
    return frames, data

def run_velocity_control_simulation(model, joint_vels, initial_pos, duration_sec, fps_val, camera_obj):
    """Run a velocity control simulation and return frames."""
    data = mujoco.MjData(model)
    mujoco.mj_resetData(model, data)
    data.qpos[:6] = initial_pos
    data.qvel[:6] = 0.0
    mujoco.mj_forward(model, data)
    
    vel_controller = VelocityController(num_joints=6)
    vel_controller.set_target_velocities(joint_vels)
    vel_controller.set_initial_positions(data.qpos[:6].copy())
    
    renderer = mujoco.Renderer(model, height=480, width=640)
    timestep = model.opt.timestep
    num_frames = int(duration_sec * fps_val)
    steps_per_frame = int(1.0 / (fps_val * timestep))
    
    frames = []
    for frame_idx in range(num_frames):
        for step in range(steps_per_frame):
            control_torque = vel_controller.compute_control(model, data)
            data.ctrl[:6] = control_torque
            mujoco.mj_step(model, data)
        
        renderer.update_scene(data, camera=camera_obj)
        frames.append(Image.fromarray(renderer.render().copy()))
    
    renderer.close()
    return frames, data

def save_gif(frames, output_path, fps_val):
    """Save frames as a GIF file."""
    frames[0].save(
        output_path,
        save_all=True,
        append_images=frames[1:],
        duration=int(1000/fps_val),
        loop=0,
        optimize=True
    )
    file_size = os.path.getsize(output_path) / (1024 * 1024)
    print(f"\nGIF saved to: {output_path} ({file_size:.2f} MB)")
    display(IPImage(filename=output_path))

print("\nHelper functions loaded successfully!")

<h1>Forward Kinematics (Position)</h1>

In [ ]:
# To be Completed by User

# Do not modify the thetas below
thetas = [
    np.random.rand()*2*np.pi-np.pi,    # Joint 0: Shoulder pan
    np.random.rand()*np.pi/2*-1-np.pi/4,   # Joint 1: Shoulder lift
    np.random.rand()*np.pi*-1+np.pi/2,    # Joint 2: Elbow
    np.random.rand()*2*np.pi-np.pi,   # Joint 3: Wrist 1
    np.random.rand()*2*np.pi-np.pi,    # Joint 4: Wrist 2
    np.random.rand()*2*np.pi-np.pi    # Joint 5: Wrist 3
]
controller.set_all_targets(thetas)
################## EDIT HERE ##################



ball_xyz = [0, -0.5, 0.5]  # [x, y, z] coordinates in meters
ball_size = 0.05  # Ball radius in meters (5cm)


###############################################

In [ ]:
# Cell 4: Forward Position Animation
model_with_ball = create_model_with_ball(model_path, ball_position=ball_xyz, ball_radius=ball_size)
frames, final_data = run_position_control_simulation(model_with_ball, controller, duration, fps, camera)

print(f"\nFinal joint positions:")
for i, pos in enumerate(final_data.qpos[:6]):
    error = controller.target_angles[i] - pos
    print(f"  Joint {i}: {pos:.4f} rad ({np.degrees(pos):.2f}°) - Error: {error:.4f} rad")

save_gif(frames, "ur10e_FK_position.gif", fps)

<h1>Inverse Kinematics (Position)</h1>

In [ ]:
# To be Completed by User

# Do not modify the ball parameters below
ball_xyz = [np.random.rand()*1.5-0.75, np.random.rand()*1.5-0.75, np.random.rand()*1]  # [x, y, z] coordinates in meters
ball_size = 0.05  # Ball radius in meters (5cm)


################## EDIT HERE ##################

thetas = [
    -np.pi/2,    # Joint 0: Shoulder pan
    -np.pi/2,   # Joint 1: Shoulder lift
    -np.pi/2,    # Joint 2: Elbow
    -np.pi/2,   # Joint 3: Wrist 1
    -np.pi/2,    # Joint 4: Wrist 2
    -np.pi/2    # Joint 5: Wrist 3
]



###############################################

controller.set_all_targets(thetas)

In [ ]:
# Cell 6: Inverse Position Animation
model_with_ball = create_model_with_ball(model_path, ball_position=ball_xyz, ball_radius=ball_size)
frames, final_data = run_position_control_simulation(model_with_ball, controller, duration, fps, camera)

print(f"\nFinal joint positions:")
for i, pos in enumerate(final_data.qpos[:6]):
    error = controller.target_angles[i] - pos
    print(f"  Joint {i}: {pos:.4f} rad ({np.degrees(pos):.2f}°) - Error: {error:.4f} rad")

save_gif(frames, "ur10e_IK_position.gif", fps)

<h1>Forward Kinematics (Velocity)</h1>

In [ ]:
# To be Completed by User

# Do not modify the initial joint angles and joint velocities below

# Set initial joint angles (radians)
initial_joint_angles = [
    -np.pi/2,    # Joint 0: Shoulder pan
    -np.pi/2,   # Joint 1: Shoulder lift
    -np.pi/2,    # Joint 2: Elbow
    -np.pi/2,   # Joint 3: Wrist 1
    -np.pi/2,    # Joint 4: Wrist 2
    -np.pi/2 # Joint 5: Wrist 3
]

# Random joint velocities (rad/s)
joint_velocities = [
    np.random.rand()*2-1,    # Joint 0 velocity
    np.random.rand()*2-1,    # Joint 1 velocity
    np.random.rand()*2-1,    # Joint 2 velocity
    0.0,    # Joint 3 velocity
    0.0,    # Joint 4 velocity
    0.0     # Joint 5 velocity
]

################## EDIT HERE ##################
# Guess the end-effector velocity direction (for arrow visualization)
# This is your prediction - the simulation will show if you're correct!


predicted_ee_velocity_guess = [
    0.3,    # vx - velocity in x direction (m/s)
    0.2,    # vy - velocity in y direction (m/s)
    0.1     # vz - velocity in z direction (m/s)
]
################################################

In [ ]:
# Cell 8: Forward Velocity Animation
# Get initial end-effector position
model_temp = mujoco.MjModel.from_xml_path(model_path)
data_temp = mujoco.MjData(model_temp)
data_temp.qpos[:6] = initial_joint_angles
mujoco.mj_forward(model_temp, data_temp)

ee_site_name = "attachment_site"
ee_site_id = mujoco.mj_name2id(model_temp, mujoco.mjtObj.mjOBJ_SITE, ee_site_name)
ee_position = data_temp.site_xpos[ee_site_id].copy()


print(f"\nJoint Velocities (rad/s):")
for i, vel in enumerate(joint_velocities):
    print(f"  Joint {i}: {vel:.4f} rad/s")

print(f"\nYour Guess for End-Effector Velocity:")
print(f"  vx = {predicted_ee_velocity_guess[0]:.4f} m/s")
print(f"  vy = {predicted_ee_velocity_guess[1]:.4f} m/s")
print(f"  vz = {predicted_ee_velocity_guess[2]:.4f} m/s")
print(f"  Speed: {np.linalg.norm(predicted_ee_velocity_guess):.4f} m/s")

# Run simulation with velocity arrow
if np.linalg.norm(predicted_ee_velocity_guess) > 1e-6:
    print(f"  → Cyan arrow shows your velocity guess")

model_with_arrow = create_model_with_velocity_arrow(model_path, ee_position, predicted_ee_velocity_guess, arrow_scale)
frames, final_data = run_velocity_control_simulation(model_with_arrow, joint_velocities, initial_joint_angles, animation_duration, animation_fps, camera)

save_gif(frames, "ur10e_FK_velocity.gif", animation_fps)

<h1>Inverse Kinematics (Velocity)</h1>

In [ ]:
# To be Completed by User

# Do not modify the initial joint angles and arrow direction below

# Set initial joint angles (radians)
initial_joint_angles = [
    -np.pi/2,    # Joint 0: Shoulder pan
    -np.pi/2,   # Joint 1: Shoulder lift
    -np.pi/2,    # Joint 2: Elbow
    -np.pi/2,   # Joint 3: Wrist 1
    -np.pi/2,    # Joint 4: Wrist 2
    -np.pi/2 # Joint 5: Wrist 3
]

# Generate random velocity arrow direction
random_direction = np.random.randn(3)
random_direction = random_direction / np.linalg.norm(random_direction)
arrow_speed = 0.4  # m/s
desired_velocity = random_direction * arrow_speed
print(desired_velocity)

################## EDIT HERE ##################
# Set joint velocities (rad/s)
joint_velocities = [
    0.5,    # Joint 0 velocity
    0.3,    # Joint 1 velocity
    -0.4,    # Joint 2 velocity
    0.0,    # Joint 3 velocity
    0.0,    # Joint 4 velocity
    0.0     # Joint 5 velocity
]
################################################



In [ ]:
# Cell 10: Inverse Velocity Animation
# Get initial end-effector position
model_temp = mujoco.MjModel.from_xml_path(model_path)
data_temp = mujoco.MjData(model_temp)
data_temp.qpos[:6] = initial_joint_angles
mujoco.mj_forward(model_temp, data_temp)

ee_site_name = "attachment_site"
ee_site_id = mujoco.mj_name2id(model_temp, mujoco.mjtObj.mjOBJ_SITE, ee_site_name)
ee_position = data_temp.site_xpos[ee_site_id].copy()

print(f"\nJoint Velocities (rad/s):")
for i, vel in enumerate(joint_velocities):
    print(f"  Joint {i}: {vel:.4f} rad/s")

print(f"\nRandom Arrow Direction:")
print(f"  vx = {desired_velocity[0]:.4f} m/s")
print(f"  vy = {desired_velocity[1]:.4f} m/s")
print(f"  vz = {desired_velocity[2]:.4f} m/s")
print(f"  Speed: {np.linalg.norm(desired_velocity):.4f} m/s")

if np.linalg.norm(desired_velocity) > 1e-6:
    print(f"  → Cyan arrow shows random velocity direction")

model_with_arrow = create_model_with_velocity_arrow(model_path, ee_position, desired_velocity, arrow_scale)
frames, final_data = run_velocity_control_simulation(model_with_arrow, joint_velocities, initial_joint_angles, animation_duration, animation_fps, camera)

save_gif(frames, "ur10e_IK_velocity.gif", animation_fps)